In [1]:
import areal_interpolation as areal
import geopandas as geopd
import pandas as pd
import argparse

In [2]:
blocks = geopd.read_file("/home/data/census/nyc/geo/blocks.geojson")
polygons = geopd.read_file("https://data.cityofnewyork.us/api/geospatial/d3qk-pfyz?method=export&format=GeoJSON")
LODES = pd.read_csv("/home/data/census/nyc/LODES/ny_od_main_JT01_2019.csv")

In [3]:
polygon_id_col = "ntacode"
crs = "EPSG:2263"

In [105]:
def count_all_jobs(blocks, polygons, LODES, polygon_id_col, crs):
    """
    Computes areally interpolated sums total number of people working inside supplied polygons
    
    Parameters
    ----------
    blocks: GeoDataFrame
        Geodataframe containing census block geometry 
    polygons: GeoDataFrame
        Polygons to which jobs data will be aggregated to
    LODES: DataFrame
        Raw LODES data for appropriate state
    polygon_id_col: str
        String identifying column name of polygon IDs
    crs: 
        Regionally appropriate planar CRS
    Returns
    ----------
    DataFrame
        Summed jobs data aggregated to polygon IDs
    """

    LODES["GEOID"] = LODES["w_geocode"]
    LODES["S000"] = LODES["S000"].astype(int)
    LODES = LODES.groupby("GEOID").agg(total_jobs = ("S000", "sum")).reset_index()

    out = areal.areal_interpolation(blocks, LODES, polygons, polygon_id_col, ["intersection_weight"], crs).reset_index()
    return out

In [106]:
def count_jobs_to_subtract(blocks, polygons, LODES, polygon_id_col, crs):
    
    weights = areal.calculate_areal_weights(polygons, blocks, "ntacode")

    polygon_x = polygon_id_col + '_x'
    polygon_y = polygon_id_col + '_y'
    
    jobs = (LODES
            .merge(t, right_on = "GEOID", left_on = "h_geocode")
            .merge(t, left_on = "w_geocode", right_on = "GEOID"))
    
    jobs_begining_ending_same_polygon = jobs[jobs[polygon_x] == jobs[polygon_y]]
    
    jobs_begining_ending_same_polygon = jobs_begining_ending_same_polygon[[polygon_x, 
                                                                           "w_geocode", 
                                                                           "h_geocode",
                                                                           "intersection_weight_x", 
                                                                           "intersection_weight_y", 
                                                                           "S000"]]
    
    jobs_begining_ending_same_polygon["full_weight"] = (jobs_begining_ending_same_polygon["intersection_weight_x"] * 
                                                        jobs_begining_ending_same_polygon["intersection_weight_y"])
    
    jobs_begining_ending_same_polygon["jobs_weighted"] = (jobs_begining_ending_same_polygon["S000"] * 
                                                          jobs_begining_ending_same_polygon["full_weight"])
    
    out = (jobs_begining_ending_same_polygon
           .groupby(polygon_x)
           .agg(jobs_to_subtract = ("jobs_weighted", "sum"))
           .reset_index()
           .rename({polygon_x : polygon_id_col}, axis = 1))
    
    return out

In [113]:
def count_jobs(blocks, polygons, LODES, polygon_id_col, crs):
    polygons = polygons.to_crs(crs)
    blocks = blocks.to_crs(crs)
    blocks = areal.calculate_census_areas(blocks)

    LODES["w_geocode"] = LODES["w_geocode"].astype(str)
    LODES["h_geocode"] = LODES["h_geocode"].astype(str)
    LODES = LODES[["w_geocode", "h_geocode", "S000"]]
    
    jobs_full = count_all_jobs(blocks, polygons, LODES, polygon_id_col, crs)
    jobs_to_subtract = count_jobs_to_subtract(blocks, polygons, LODES, polygon_id_col, crs)
    
    jobs_merged = jobs_full.merge(jobs_to_subtract)
    jobs_merged["jobs"] = jobs_merged["total_jobs"] - jobs_merged["jobs_to_subtract"]
    
    return jobs_merged[[polygon_id_col, "jobs"]]

In [114]:
count_jobs(blocks, polygons, LODES, polygon_id_col, crs)

,ntacode,jobs
0,BK09,14317.689717
1,BK17,16287.568370
2,BK19,17249.963131
3,BK21,5248.983159
4,BK23,3010.496465
...,...,...
189,SI37,4311.745158
190,SI45,5361.828506
191,SI48,1203.583974
192,SI54,4991.206734


In [ ]:
jobs_begining_ending_same_polygon.head(20)

,ntacode_x,w_geocode,h_geocode,intersection_weight_x,intersection_weight_y,S000,full_weight
26,BX34,360050043002002,360050051003000,2.781583e-04,0.000228,1,6.349574e-08
29,BX39,360050043002002,360050051003000,9.997218e-01,0.999772,1,9.994936e-01
46,BX34,360050043002002,360050075003000,1.000000e+00,0.000228,1,2.282719e-04
82,BX34,360050043002002,360050035002000,1.069475e-06,0.000228,1,2.441311e-10
85,BX39,360050043002002,360050035002000,9.999989e-01,0.999772,1,9.997707e-01
117,BX39,360050043002002,360050019003016,1.000000e+00,0.999772,1,9.997717e-01
185,BX39,360050043002002,360050025003000,1.000000e+00,0.999772,2,9.997717e-01
187,BX39,360050043002002,360050027012000,1.000000e+00,0.999772,2,9.997717e-01
315,BX39,360050043002002,360050027021002,1.000000e+00,0.999772,1,9.997717e-01
356,BX34,360050043002002,360050067005000,1.000000e+00,0.000228,1,2.282719e-04
